In [3]:
import numpy as np
import snake_env, snake_env2

In [4]:
env1 = snake_env.Snake(grid_size=(8,8))
env2 = snake_env2.Snake(grid_size=(8,8), body_length=5)

In [5]:
env1.render(mode='char')

░░░░░░░░░░░░░░░░
██████░░░░░░░░░░
░░████░░○○░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░


In [6]:
env2.render(mode='char')

░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░
░░░░░░████░░○○░░
░░░░░░████░░░░░░
░░░░░░██░░░░░░░░


In [25]:
import time
from functools import wraps

def timer(func):
    @wraps(func)
    def timer_wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Func: {func.__name__}{args} {kwargs} took {end - start:.4f} sec')
        return res
    return timer_wrapper

@timer
def play(env: snake_env.Snake):
    for i in range(10000):
        done, info = False, {}
        reward = 0
        obs = env.reset()
        while not done:
            obs, reward, done, info = env.step(np.random.randint(0, 4))

NameError: name 'snake_env' is not defined

In [18]:
play(env1)

Func: play(<snake_env.Snake object at 0x7f91e01f07c0>,) {} took 2.0154 sec


In [19]:
play(env2)

Func: play(<snake_env2.Snake object at 0x7f91e01f02e0>,) {} took 4.3828 sec


In [11]:
from stable_baselines3.common.utils import set_random_seed

def random_play(env):
    set_random_seed(42, using_cuda=False)
    trial_rewards = []
    for _ in range(1000):
        done, info = False, {}
        reward = 0
        obs = env.reset()
        cum_reward = 0
        while not done:
            action = np.random.randint(0, 4)
            obs, reward, done, info = env.step(action)
            cum_reward += reward
        trial_rewards.append(cum_reward)
    
    return np.mean(trial_rewards)

In [12]:
r1 = random_play(env1)
r2 = random_play(env2)

In [13]:
r1

-1.0011110990609435

In [14]:
r2

-4.995340592643895

In [22]:
import numpy as np

direction_vec = np.array([[-1,0], [0,1], [1,0], [0,-1]])
max_shape = 8

def random_body(length = 5):
    board = np.zeros((max_shape, max_shape))
    body = [(np.random.randint(1, max_shape - 1), np.random.randint(1, max_shape - 1))]
    board[body[0]] = 1

    def get_candidate(pos):
        candidate = pos + direction_vec
        candidate = candidate[(candidate >= 0).all(axis=1) & (candidate < max_shape).all(axis=1)]
        candidate = candidate[board[candidate[:,0], candidate[:,1]] == 0]
        candidate = [tuple(x) for x in candidate]
        return candidate

    for _ in range(length - 1):
        head_candidate = get_candidate(body[0])
        if len(body) == 1:
            body_candidate = head_candidate
        else:
            body_candidate = get_candidate(body[-1])
        
        if len(head_candidate) == 1 and head_candidate[0] in body_candidate:
            body_candidate.remove(head_candidate[0])

        if len(body_candidate) == 0:
            break

        body.append(body_candidate[np.random.randint(0, len(body_candidate))])
        board[body[-1]] = 1
    
    return body


body = random_body(9)
print(body)
print('  ' + ' '.join(list(map(str, range(max_shape)))))
for i in range(max_shape):
    print(i, end=' ')
    for j in range(max_shape):
        if (i,j) in body:
            print('x', end=' ')
        else:
            print('.', end=' ')
    print()


[(3, 3), (3, 2), (4, 2), (4, 3), (4, 4), (3, 4), (2, 4), (1, 4), (0, 4)]
  0 1 2 3 4 5 6 7
0 . . . . x . . . 
1 . . . . x . . . 
2 . . . . x . . . 
3 . . x x x . . . 
4 . . x x x . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 


In [26]:
import time
from functools import wraps

def timer(func):
    @wraps(func)
    def timer_wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Func: {func.__name__}{args} {kwargs} took {end - start:.4f} sec')
        return res
    return timer_wrapper

In [28]:
@timer
def test(i=10000):
    body = random_body(9)

test()


Func: test() {} took 0.0009 sec


In [29]:
(0,1) + np.array([1,0])

array([1, 1])